In [ ]:
import pandas as pd         # 导入pandas库，它就像一个超级强大的Excel工具，专门用来处理表格数据（DataFrame）。

# 加载数据集并指定编码为gbk
data = pd.read_csv('medical_data.csv',encoding='gbk')
# 使用pandas的read_csv函数，读取名为 'medical_data.csv' 的CSV文件。
# encoding='gbk' 参数是用来指定文件的编码格式为 'gbk'。
# 这是因为中文系统下创建的CSV文件可能不是默认的 'utf-8' 编码，而是 'gbk' 或 'gb2312'，
# 指定正确的编码可以避免中文乱码问题。
# 将文件内容加载到一个名为 'data' 的DataFrame（可以理解为表格）中。

# 查看数据类型
print(data.dtypes)          # 打印出DataFrame中每一列的名称及其对应的数据类型（例如，int64、float64、object等）。
                            # 了解数据类型对于后续的数据清洗和分析至关重要。
# 查看表结构基本信息
print(data.info())          # 打印DataFrame的概要信息，包括每列的非空值数量和数据类型，
                            # 以及内存使用情况。这比 `dtypes` 更详细，可以快速识别哪些列可能存在缺失值。

# 显示每一列的空缺值数量
print(data.isnull().sum())  # `data.isnull()` 会创建一个与原DataFrame形状相同，但内容为True/False的DataFrame，
                            # 其中True表示原始位置有缺失值（NaN）。
                            # `.sum()` 会对这个True/False DataFrame进行求和，因为True被算作1，False被算作0，
                            # 所以结果就是每一列缺失值的总数量。这有助于我们了解缺失值的分布情况。

# 规范日期格式
data['就诊日期'] = pd.to_datetime(data['就诊日期'])
# 将 '就诊日期' 列转换为pandas的日期时间（datetime）格式。
# `pd.to_datetime()` 函数能够灵活地解析多种日期字符串格式。
# 这样做的好处是，可以让日期数据进行日期计算（例如，计算天数差）和更方便的筛选。
data['诊断日期'] = pd.to_datetime(data['诊断日期'])
# 同样地，将 '诊断日期' 列转换为日期时间格式。

# 修改列名
data.rename(columns={'病人ID': '患者ID'}, inplace=True)
# 使用DataFrame的 .rename() 方法来修改列名。
# `columns={'旧列名': '新列名'}` 是一个字典，指定了要修改的列名映射关系。
# `inplace=True` 表示直接在原始DataFrame 'data' 上进行修改，而不是返回一个新的DataFrame。

# 查看修改后的表结构
print(data.head())          # 再次显示数据的前5行，确认列名 '病人ID' 是否已成功修改为 '患者ID'。

from datetime import datetime # 从Python的内置 `datetime` 模块中导入 `datetime` 类，用于处理日期和时间。

# 增加诊断延迟和病程列
data['诊断延迟'] = (data['诊断日期'] - data['就诊日期']).dt.days
# 新增一列名为 '诊断延迟'。
# `data['诊断日期'] - data['就诊日期']`：由于这两列已经转换为日期时间格式，可以直接相减，结果是一个时间差（Timedelta）Series。
# `.dt.days`：从时间差Series中提取出天数部分。
# 这样，'诊断延迟' 列就表示从就诊到诊断之间相隔的天数。
data['病程'] = (datetime(2024, 9, 1) - data['诊断日期']).dt.days
# 新增一列名为 '病程'。
# `datetime(2024, 9, 1)`：定义一个固定的参考日期（这里是2024年9月1日）。
# `datetime(...) - data['诊断日期']`：计算参考日期与每个病人的诊断日期之间的时间差。
# `.dt.days`：提取出天数部分。这样，'病程' 列表示从诊断日期到2024年9月1日之间相隔的天数。

# 删除不合理的数据
data = data[(data['诊断延迟'] >= 0) & (data['年龄'] > 0) & (data['年龄'] < 120)]
# 这一行代码是进行数据筛选，删除不符合逻辑或不合理的数据记录。
# `(data['诊断延迟'] >= 0)`：筛选出 '诊断延迟' 不为负数的行（诊断日期应该不早于就诊日期）。
# `(data['年龄'] > 0)`：筛选出年龄大于0的行（年龄不能为0或负数）。
# `(data['年龄'] < 120)`：筛选出年龄小于120的行（通常将大于某个极端值的年龄视为不合理或录入错误）。
# `&`：是逻辑“与”操作符，表示必须同时满足所有这些条件，行才会被保留。
# 通过这些条件，我们移除了数据中一些明显错误或不符合实际情况的记录。

# 查看修改后的数据
print(data.describe())      # 打印DataFrame的描述性统计信息。
                            # 这包括了数值列的计数、均值、标准差、最小值、25%分位数(Q1)、中位数(50%分位数/Q2)、
                            # 75%分位数(Q3)和最大值。这有助于我们快速了解数据分布和特征的统计特性。

# 删除重复值并记录删除的行数
initial_rows = data.shape[0]        # 记录处理前的数据总行数。
data.drop_duplicates(inplace=True)  # 使用DataFrame的 .drop_duplicates() 方法，删除DataFrame中所有完全相同的重复行。
                                    # `inplace=True` 表示直接在原始DataFrame 'data' 上修改。
deleted_rows = initial_rows - data.shape[0] # 计算删除了多少行：初始行数减去处理后的行数。

print(f'删除的重复行数: {deleted_rows}') # 打印出总共删除了多少重复行。

from sklearn.preprocessing import MinMaxScaler # 从scikit-learn库中导入MinMaxScaler，这是一个用于数据归一化的工具。

# 对需要归一化的列进行处理
scaler = MinMaxScaler()             # 创建一个MinMaxScaler的实例（一个“归一化工具”）。
columns_to_normalize = ['年龄', '体重', '身高'] # 定义一个列表，包含我们想要进行归一化处理的列名。
data[columns_to_normalize] = scaler.fit_transform(data[columns_to_normalize])
# data[columns_to_normalize]：选取DataFrame `data` 中所有需要归一化的列。
# scaler.fit_transform(...)：这是MinMaxScaler的核心方法。它做了两件事：
#   1. `fit`：在选定列中找到每个列的最小值（min）和最大值（max）。
#   2. `transform`：对选定列中的每个值进行归一化处理，将数据缩放到0到1之间。
#                     具体公式是：(原始值 - 最小值) / (最大值 - 最小值)。
# 归一化有助于消除不同特征之间的量纲影响，确保所有特征在模型训练时具有相同的权重。
# 将归一化后的结果重新赋值回 `data` 的相应列中。

# 查看归一化后的数据
print(data.head())                  # 再次显示数据的前5行，检查 '年龄'、'体重'、'身高' 列的值是否已被归一化到0-1之间。

import matplotlib.pyplot as plt     # 导入matplotlib.pyplot库，主要用于创建静态、动态交互式的可视化图表。
import matplotlib.font_manager as fm # 导入matplotlib.font_manager库，用于管理和查找字体，特别是处理中文字体。


# 统计治疗结果分布
treatment_outcome_distribution = data.groupby('疾病类型')['治疗结果'].value_counts().unstack()
# `data.groupby('疾病类型')`：按 '疾病类型' 列对数据进行分组。
# `['治疗结果'].value_counts()`：在每个疾病类型组内，统计 '治疗结果' 列中每个值的出现次数。
# `.unstack()`：将多级索引的Series转换为DataFrame，使 '治疗结果' 的不同类别成为列。
# 这样，`treatment_outcome_distribution`就会是一个DataFrame，行是疾病类型，列是治疗结果的种类，值为每种组合的数量。

# 设置中文字体
font_path = 'C:/Windows/Fonts/simhei.ttf'  # 根据你的系统调整字体路径
# 定义一个变量 `font_path`，存储系统中常用中文字体（例如“黑体”）的文件路径。
# 注意：这个路径在不同操作系统或不同用户环境下可能需要调整。
my_font = fm.FontProperties(fname=font_path)
# 使用 `matplotlib.font_manager.FontProperties` 创建一个字体属性对象，
# 指定使用 `font_path` 指向的字体文件，以便matplotlib正确显示中文。

# 绘制柱状图
treatment_outcome_distribution.plot(kind='bar', stacked=True)
# 使用DataFrame的内置 `plot()` 方法绘制柱状图。
# `kind='bar'` 指定图表类型为柱状图。
# `stacked=True` 表示柱子是堆叠的，即不同治疗结果的计数在每个疾病类型的柱子上堆叠显示。
plt.title('不同疾病类型的治疗结果分布', fontproperties=my_font)
# 设置图表的标题，并使用之前定义好的 `my_font` 来显示中文标题。
plt.xlabel('疾病类型', fontproperties=my_font)
# 设置x轴标签，并使用 `my_font` 显示中文。
plt.ylabel('治疗结果数量', fontproperties=my_font)
# 设置y轴标签，并使用 `my_font` 显示中文。
plt.xticks(fontproperties=my_font)  # 设置x轴刻度标签的字体
# 同样地，设置x轴上的刻度标签（例如疾病类型的名称）的字体为 `my_font`，以确保中文正常显示。
plt.yticks(fontproperties=my_font)  # 设置y轴刻度标签的字体
# 设置y轴上的刻度标签（例如数量）的字体为 `my_font`。
plt.legend(prop=my_font)  # 设置图例字体
# 设置图例（区分不同治疗结果颜色的方块和文本）的字体为 `my_font`。
plt.show()                          # 显示绘制好的柱状图。

# 绘制散点图
plt.scatter(data['年龄'], data['疾病严重程度'])
# 使用 `plt.scatter()` 函数绘制散点图。
# `data['年龄']` 作为x轴数据，`data['疾病严重程度']` 作为y轴数据。
plt.title('年龄和疾病严重程度的关系', fontproperties=my_font)
# 设置散点图的标题。
plt.xlabel('年龄', fontproperties=my_font)
# 设置x轴标签。
plt.ylabel('疾病严重程度', fontproperties=my_font)
# 设置y轴标签。
plt.xticks(fontproperties=my_font)  # 设置x轴刻度标签的字体
# 设置x轴刻度标签的字体。
plt.yticks(fontproperties=my_font)  # 设置y轴刻度标签的字体
# 设置y轴刻度标签的字体。
plt.legend(prop=my_font)  # 设置图例字体
# 注意：这个散点图没有明确的图例，`plt.legend()` 在此处可能不会显示任何内容，
# 但如果后续添加了多个散点系列，`prop=my_font` 会起作用。
plt.show()                          # 显示绘制好的散点图。

# 保存处理后得数据
output_path = '2.1.4_cleaned_data.csv' # 定义保存文件的路径和名称。
data.to_csv(output_path, index=False)
# `data.to_csv(...)`：这是pandas用来将DataFrame保存为CSV文件的方法。
# `output_path`：指定新保存的文件的路径和名称。
# `index=False`：这个参数非常重要，它表示在保存CSV文件时，不要把DataFrame的索引（左边默认从0开始的数字序号）也写入到CSV文件中。
# 这样做可以避免文件中有额外且无用的列。
